# **Primer on Working With Text Data**

Let's start off with checking versions of key libraries, in case these notebooks need to be revisited in the future and changes caused by updates to the underlying toolset are easier to track.

In [1]:
from importlib.metadata import version
print("torch version", version("torch"))
print("tiktoken version", version("tiktoken"))

torch version 2.7.0
tiktoken version 0.9.0


In [2]:
import os, re
import urllib.request


## **1. Tokenizing Text**

In the book, Sebastian demonstrates the tokenization of text using Edith Wharton's "The Verdict" as an example. I'll be using his provided example as well as samples from my own collection. Starting off with Frederic Bastiat's "The Law", which was published in the 19th century, offers a very different style of writing and punctuation compared to more modern writers.

In [3]:
# Downloading sample text
if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path  = "data/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [4]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    verdict = f.read()

print("Total number of characters: ", len(verdict))
print(verdict[:99])

Total number of characters:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
# And now for Bastiat's text.
with open("data/the-law-bastiat.txt", "r", encoding="utf-8") as f:
    law = f.read()

print("Total number of characters: ", len(law))
print(law[:99])

Total number of characters:  95988
 The law perverted! The law—and, in its wake, all the collective forces of the nation—the law, I sa


These texts will be tokenized and embedded in an LLM. Initially, a simple tokenizer will be created in the next few cells and then it can be applied to the texts.

In [6]:
# This regex splits on white spaces.
test = "In the grim darkness of the far future, there is only war!"
result = re.split(r'(\s)', test)
print(result)

['In', ' ', 'the', ' ', 'grim', ' ', 'darkness', ' ', 'of', ' ', 'the', ' ', 'far', ' ', 'future,', ' ', 'there', ' ', 'is', ' ', 'only', ' ', 'war!']


In [7]:
# The regex should also be able to split on punctuations.
result = re.split(r'([,.!]|\s)', test)
print(result)

['In', ' ', 'the', ' ', 'grim', ' ', 'darkness', ' ', 'of', ' ', 'the', ' ', 'far', ' ', 'future', ',', '', ' ', 'there', ' ', 'is', ' ', 'only', ' ', 'war', '!', '']


In [8]:
# Now we can strip whitespaces from each item while filtering out the empty strings
result = [item for item in result if item.strip()]
print(result)

['In', 'the', 'grim', 'darkness', 'of', 'the', 'far', 'future', ',', 'there', 'is', 'only', 'war', '!']


It should be noted that stripping whitespaces can aid in brevity, while also reducing memory requirements, especially when working with LLMS using consumer hardware. However, this practice is usually avoided when the retention of exact sentence structures is important for e.g. working with programming languages where indentation and spacing are important.

In [9]:
# Also accounting for other punctuation types
test = "Hello, world! Is this a test? Surely--it must be."
result = re.split(r'([,./:"?_()\']|--|\s)', test)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world!', 'Is', 'this', 'a', 'test', '?', 'Surely', '--', 'it', 'must', 'be', '.']


Time to test this tokenization on the text samples.

In [10]:
verdict_preproc = re.split(r'([,./:"?!_()\']|--|\s)', verdict)
law_preproc = re.split(r'([,./:"?!_()\']|--|\s)', law)

verdict_preproc = [item.strip() for item in verdict_preproc if item.strip()]
law_preproc = [item.strip() for item in law_preproc if item.strip()]

In [11]:
print(verdict_preproc[:99])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '--', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '--', 'his', 'last', 'Chicago', 'sitter']


In [12]:
print(law_preproc[:99])

['The', 'law', 'perverted', '!', 'The', 'law—and', ',', 'in', 'its', 'wake', ',', 'all', 'the', 'collective', 'forces', 'of', 'the', 'nation—the', 'law', ',', 'I', 'say', ',', 'not', 'only', 'diverted', 'from', 'its', 'proper', 'direction', ',', 'but', 'made', 'to', 'pursue', 'one', 'entirely', 'contrary', '!', 'The', 'law', 'become', 'the', 'tool', 'of', 'every', 'kind', 'of', 'avarice', ',', 'instead', 'of', 'being', 'its', 'check', '!', 'The', 'law', 'guilty', 'of', 'that', 'very', 'iniquity', 'which', 'it', 'was', 'its', 'mission', 'to', 'punish', '!', 'Truly', ',', 'this', 'is', 'a', 'serious', 'fact', ',', 'if', 'it', 'exists', ',', 'and', 'one', 'to', 'which', 'I', 'feel', 'bound', 'to', 'call', 'the', 'attention', 'of', 'my', 'fellow', 'citizens', '.']


In [13]:
# Total number of tokens for each text.
print(f"Tokens created:\nThe Verdict: {len(verdict_preproc)}\nThe Law: {len(law_preproc)}")

Tokens created:
The Verdict: 4669
The Law: 18807


## **2. Converting Tokens Into Token IDs**

The next step involves the conversion of text tokens into token IDs which can be processed via embedding layers.

In [14]:
all_words_verdict = sorted(set(verdict_preproc))
all_words_law = sorted(set(law_preproc))

vocab_sz_verdict = len(all_words_verdict)
vocab_sz_law = len(all_words_law)

print(f"Vocab Size:\nThe Verdict: {vocab_sz_verdict}\nThe Law: {vocab_sz_law}")

Vocab Size:
The Verdict: 1143
The Law: 3152


In [15]:
def create_token_ids(x):
    return {token:integer for integer, token in enumerate(x)}

vocab_verdict = create_token_ids(all_words_verdict)
vocab_law = create_token_ids(all_words_law)

In [16]:
# Displaying a sample containing token ids
for i, item in enumerate(vocab_verdict.items()):
    print(item)
    if i >= 50: 
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Carlo;', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('Florence', 35)
('For', 36)
('Gallery', 37)
('Gideon', 38)
('Gisburn', 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
('Grindles', 44)
('HAD', 45)
('Had', 46)
('Hang', 47)
('Has', 48)
('He', 49)
('Her', 50)


`Experiment Note` strip "The Law" of ids linked to reference markers. Also, be mindful that integers are also being used to refer to dates.

Moving onto creating a simple tokenizer which encodes text into token IDs and also turns token IDs back into text.

In [37]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_2_int = vocab
        self.int_2_str = {i:s for s, i in vocab.items()}

    def encode(self, target):
        preprocessed = re.split(r'([,./:"?!_()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_2_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_2_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [41]:
# Testing the tokenizer using The Law's vocab.
t1 = SimpleTokenizerV1(vocab_law)

text = """If you suggest a doubt as to the morality of these institutions, it is said directly—
          "You are a dangerous experimenter,a utopian, a theorist, a despiser of the laws; 
          you would shake the basis upon which society rests!"""
id1 = t1.encode(text)
print(id1)

[184, 3142, 2810, 362, 1074, 541, 2912, 2872, 1973, 2063, 2885, 1697, 5, 1746, 1741, 2586, 1012, 1, 360, 515, 362, 917, 1268, 5, 362, 3007, 5, 362, 2881, 5, 362, 977, 2063, 2872, 1796, 3142, 3133, 2667, 2872, 612, 2999, 3082, 2713, 2532, 0]


In [44]:
# Testing the tokenizer using The Verdict's vocab
t2 = SimpleTokenizerV1(vocab_verdict)

text = """It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
id2 = t2.encode(text)
print(id2)

[57, 2, 861, 999, 610, 538, 754, 5, 1139, 603, 5, 1, 68, 7, 39, 862, 1121, 764, 803, 7]


Note that unique tokenizers are being used for the two texts since the texts contain words which don't overlap with one-another.

Let's decode the IDs back to text.

In [45]:
t1.decode(id1)

'If you suggest a doubt as to the morality of these institutions, it is said directly—" You are a dangerous experimenter, a utopian, a theorist, a despiser of the laws; you would shake the basis upon which society rests!'

In [46]:
t2.decode(id2)

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## **3. Adding Special Context Tokens**

## **4. BytePair Encoding**

## **5. Data Sampling With a Sliding Window**

## **6. Creating Token Embeddings**

## **7. Encoding Word Positions**